In [ ]:
%pylab inline

In [ ]:
# todo clean this up

import os
import sys
import time
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pickle



# for testing
import random

In [ ]:
# Root directory of the project
ROOT_DIR = os.path.abspath("../../")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn.config import Config
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.model import log

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

In [ ]:
class NIAPConfig(Config):
    """Configuration for training on the toy shapes dataset.
    Derives from the base Config class and overrides values specific
    to the toy shapes dataset.
    """
    # Give the configuration a recognizable name
    NAME = "niap"

    # Train on 1 GPU and 4 images per GPU. We can put multiple images on each
    # GPU because the images are small. Batch size is 8 (GPUs * images/GPU).
    GPU_COUNT = 1
    IMAGES_PER_GPU = 2

    # Number of classes (including background)
    NUM_CLASSES = 1 + 1  # background + 1 shapes

    
    DETECTION_MIN_CONFIDENCE = 0
    
    # Use small images for faster training. Set the limits of the small side
    # the large side, and that determines the image shape.
    IMAGE_MIN_DIM = 320
    IMAGE_MAX_DIM = 640

    # Use smaller anchors because our image and objects are small
    RPN_ANCHOR_SCALES = (16, 32, 64, 128, 256)  # anchor side in pixels

    # Reduce training ROIs per image because the images are small and have
    # few objects. Aim to allow ROI sampling to pick 33% positive ROIs.
    TRAIN_ROIS_PER_IMAGE = 100

    #Integer. Total number of steps (batches of samples) to yield from generator 
    #before declaring one epoch finished and starting the next epoch. 
    #It should typically be equal ceil(num_samples / batch_size).
    STEPS_PER_EPOCH = 4493
    
    LEARNING_RATE = 0.00001  # defalt is 0.001

    # use small validation steps since the epoch is small
    VALIDATION_STEPS = 8
    
    LEARNING_MOMENTUM  = 0.9 # default is 0.9
    
config = NIAPConfig()
config.display()

In [ ]:
# todo move to NIAP FISH Dataset

def generate_masks(points, size=(720, 1280), dtype=bool):
    '''
    get the annotation points then generate mask image
    
    input
        points: list of annotation points
    
    output
        binary bask image
    '''
    polys = []
    polys.append(points)
    
    mask = np.zeros(size)
    
    
    cv2.fillPoly(mask, polys, 1)
    mask = mask.astype(dtype)
    
    return mask

In [ ]:
# custom class to load data

class NIAP_Fish_Dataset(utils.Dataset):


    def load_fish_dataset(self, mr_data):
        """
        niap parsed annotation 
        """
        self.mr_data = mr_data
        image_id_count = 0
        
        source = "niap"
        
        #for i, c in enumerate(classlist):
        #    self.add_class(source, 1, c)
        
        # TOTO hard coded fish class
        self.add_class(source, 1, 'fish')

        for iid in self.mr_data.keys():
            self.add_image(source, image_id=iid, path=self.mr_data[iid]['path'])
       
    
    def image_reference(self, image_id):
        info = self.image_info[image_id]
        return info['path']

    # done
    def load_image(self, image_id):
        info = self.image_info[image_id]
        return cv2.imread(info["path"])

    # todo need to change
    def load_mask(self, image_id):
        
        info = self.image_info[image_id]
        
        masks = []
        
        # hard coded since we only have fish class
        class_ids = []
        
        for i in range(len(self.mr_data[image_id]['points'])):
            m = generate_masks(self.mr_data[image_id]['points'][i])
            masks.append(m)
            
            class_ids.append(1)
        
        # class names is a list of strings e.g ['BG', 'square', 'circle', 'triangle']
        
        
        class_ids = np.array(class_ids).astype(np.int32)
        
        # TODO should do this at the beginning rather than create masks then convert to numpy array
        # mask shape is [h, w, depth]
        
        masks = np.array(masks)
        masks_np = np.zeros((masks.shape[1], masks.shape[2], masks.shape[0]))
        for i, m in enumerate(masks):
            masks_np[:, :, i] = m
        
        masks_np = masks_np.astype(np.bool)
        return masks_np, class_ids
        

In [ ]:
with open('niap_fix_maskrcnn_train0_30.pik', 'rb') as f:
    train_data = pickle.load(f)
    
with open('niap_fix_maskrcnn_val_30_40.pik', 'rb') as f:
    valid_data = pickle.load(f)

with open('niap_fix_maskrcnn_test_40_45.pik', 'rb') as f:
    test_data = pickle.load(f)

In [ ]:
len(train_data), len(valid_data), len(test_data)

In [ ]:
# Training dataset
dataset_train = NIAP_Fish_Dataset()

dataset_train.load_fish_dataset(train_data)
dataset_train.prepare()

# Validation dataset
dataset_val = NIAP_Fish_Dataset()
dataset_val.load_fish_dataset(valid_data)
dataset_val.prepare()


# Testing dataset
dataset_test = NIAP_Fish_Dataset()
dataset_test.load_fish_dataset(test_data)
dataset_test.prepare()

In [ ]:
# Create model in training mode
model = modellib.MaskRCNN(mode="training", config=config,
                          model_dir=MODEL_DIR)

In [ ]:
# Which weights to start with?
init_with = "coco"  # imagenet, coco, or last

if init_with == "imagenet":
    model.load_weights(model.get_imagenet_weights(), by_name=True)
elif init_with == "coco":
    # Load weights trained on MS COCO, but skip layers that
    # are different due to the different number of classes
    # See README for instructions to download the COCO weights
    model.load_weights(COCO_MODEL_PATH, by_name=True,
                       exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", 
                                "mrcnn_bbox", "mrcnn_mask"])
elif init_with == "last":
    # Load the last model you trained and continue training
    model.load_weights(model.find_last(), by_name=True)

In [ ]:
model.keras_model.metrics_tensors = []

In [ ]:
# Train the head branches
# Passing layers="heads" freezes all layers except the head
# layers. You can also pass a regular expression to select
# which layers to train by name pattern.
#model.train(dataset_train, dataset_val, 
#            learning_rate=config.LEARNING_RATE, 
#            epochs=10, 
#            layers='heads')

In [ ]:
# Fine tune all layers
# Passing layers="all" trains all layers. You can also 
# pass a regular expression to select which layers to
# train by name pattern.
model.train(dataset_train, dataset_val, 
            learning_rate=config.LEARNING_RATE,
            epochs=50, 
            layers="all")

In [ ]:
model.train(dataset_train, dataset_val, 
            learning_rate=config.LEARNING_RATE/10,
            epochs=80, 
            layers="all")

In [ ]:
model.train(dataset_train, dataset_val, 
            learning_rate=config.LEARNING_RATE/10,
            epochs=120, 
            layers="all")

In [ ]:
model.train(dataset_train, dataset_val, 
            learning_rate=config.LEARNING_RATE/100,
            epochs=150, 
            layers="all")

In [ ]:
# Save weights
# Typically not needed because callbacks save after every epoch
# Uncomment to save manually
#model_path = os.path.join(MODEL_DIR, "mask_rcnn_niap_all_layer_l00001_e50.h5")
#model.keras_model.save_weights(model_path)

# exaple of extract fish from a built model #

In [ ]:
class InferenceConfig(NIAPConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

inference_config = InferenceConfig()

# Recreate the model in inference mode
model = modellib.MaskRCNN(mode="inference", 
                          config=inference_config,
                          model_dir=MODEL_DIR)



In [ ]:
# Get path to saved weights
# Either set a specific path or find last trained weights
model_path = os.path.join(MODEL_DIR, "niap20200319T1356/mask_rcnn_niap_0074.h5")
#model_path = model.find_last()

# Load trained weights
print("Loading weights from ", model_path)
model.load_weights(model_path, by_name=True)

#Loading weights from  /share/Mask_RCNN-master/logs/niap20200319T1356/mask_rcnn_niap_0074.h5
#Re-starting from epoch 74


In [ ]:
# Compute VOC-Style mAP @ IoU=0.5
# Running on 10 images. Increase for better accuracy.
image_ids = np.random.choice(dataset_val.image_ids, 5)
APs = []
for image_id in image_ids:
    # Load image and ground truth data
    image, image_meta, gt_class_id, gt_bbox, gt_mask =\
        modellib.load_image_gt(dataset_val, inference_config,
                               image_id, use_mini_mask=False)
    molded_images = np.expand_dims(modellib.mold_image(image, inference_config), 0)
    # Run object detection
    results = model.detect([image], verbose=0)
    r = results[0]
    # Compute AP
    AP, precisions, recalls, overlaps =\
        utils.compute_ap(gt_bbox, gt_class_id, gt_mask,
                         r["rois"], r["class_ids"], r["scores"], r['masks'])
    APs.append(AP)
    
print("mAP: ", np.mean(APs))

In [ ]:
dataset_val.image_ids

In [ ]:
start_time = time.time()
APs = []
for image_id in dataset_val.image_ids:
    # Load image and ground truth data
    image, image_meta, gt_class_id, gt_bbox, gt_mask =\
        modellib.load_image_gt(dataset_val, inference_config,
                               image_id, use_mini_mask=False)
    molded_images = np.expand_dims(modellib.mold_image(image, inference_config), 0)
    # Run object detection
    results = model.detect([image], verbose=0)
    r = results[0]
    # Compute AP
    AP, precisions, recalls, overlaps =\
        utils.compute_ap(gt_bbox, gt_class_id, gt_mask,
                         r["rois"], r["class_ids"], r["scores"], r['masks'])
    APs.append(AP)
    
print("validation mAP: ", np.mean(APs))

print("time used: {}s".format(time.time() - start_time))

In [ ]:
start_time = time.time()
APs = []
for image_id in dataset_test.image_ids:
    # Load image and ground truth data
    image, image_meta, gt_class_id, gt_bbox, gt_mask =\
        modellib.load_image_gt(dataset_test, inference_config,
                               image_id, use_mini_mask=False)
    molded_images = np.expand_dims(modellib.mold_image(image, inference_config), 0)
    # Run object detection
    results = model.detect([image], verbose=0)
    r = results[0]
    # Compute AP
    AP, precisions, recalls, overlaps =\
        utils.compute_ap(gt_bbox, gt_class_id, gt_mask,
                         r["rois"], r["class_ids"], r["scores"], r['masks'])
    APs.append(AP)
    
print("test mAP: ", np.mean(APs))

print("time used: {}s".format(time.time() - start_time))

In [ ]:
plt.plot(APs)

In [ ]:
from mrcnn import visualize

In [ ]:
visualize.display_instances()

In [ ]:
def get_ax(rows=1, cols=1, size=8):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Change the default size attribute to control the size
    of rendered images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

In [ ]:
#results = model.detect([original_image], verbose=1)

r = results[0]
visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                            dataset_val.class_names, r['scores'], ax=get_ax())

In [ ]:
plt.figure(figsize=(16,16))
plt.imshow(image)